In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import bz2
import pandas as pd

from datetime import datetime
from io import BufferedWriter
from elasticsearch import Elasticsearch
from tqdm.autonotebook import tqdm
from utils.elasticsearch_utils import ElasticSearchUtils
from utils.naver_reply_utils import NaverNewsReplyUtils

/home/.local/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


# 데이터 덤프

In [5]:
index = {
    'detail': [
        'crawler-naver-kin-detail',
        'crawler-naver-kin-baseball',
    ],
    'list': [
        'crawler-naver-kin-question_list_done',
    ]
}

sources = {
    'list': ['fullDirNamePath', 'document_id']
}

In [6]:
def dump_data(index_list, sources):
    """뉴스 본문이나 댓글을 덤프 받는다."""
    elastic = ElasticSearchUtils(
        host='https://corpus.ncsoft.com:9200',
        http_auth='elastic:nlplab',
    )

    query = {}
    if sources is not None:
        query['_source'] = sources
    
    result = []
    for index in index_list:
        elastic.dump_data(
            index=index,
            query=query,
            result=result,
        )

    return result

kin_list = dump_data(index_list=index['list'], sources=sources['list'])

In [10]:
kin_list_df = pd.DataFrame(kin_list)
kin_list_df = kin_list_df.dropna()
kin_list_df.head()

,document_id,fullDirNamePath
2217437,4-40601-313974322,"Q&A > 경제 > 직업, 취업 > 공무원"
2217438,4-40611-313971227,"Q&A > 경제 > 직업, 취업 > 아르바이트"
2217439,4-40611-313968280,"Q&A > 경제 > 직업, 취업 > 아르바이트"
2217440,4-40601-313988116,"Q&A > 경제 > 직업, 취업 > 공무원"
2217441,4-40611-314001886,"Q&A > 경제 > 직업, 취업 > 아르바이트"


In [58]:
kin_list_df['category_id'] = kin_list_df['document_id'].apply(lambda x: x.split('-')[1])
kin_list_df['category_top_id'] = kin_list_df['document_id'].apply(lambda x: x.split('-')[0])
kin_list_df.head()

,document_id,fullDirNamePath,category_id,category_top_id
2217437,4-40601-313974322,"Q&A > 경제 > 직업, 취업 > 공무원",40601,4
2217438,4-40611-313971227,"Q&A > 경제 > 직업, 취업 > 아르바이트",40611,4
2217439,4-40611-313968280,"Q&A > 경제 > 직업, 취업 > 아르바이트",40611,4
2217440,4-40601-313988116,"Q&A > 경제 > 직업, 취업 > 공무원",40601,4
2217441,4-40611-314001886,"Q&A > 경제 > 직업, 취업 > 아르바이트",40611,4


In [60]:
category_count = kin_list_df.groupby(by=['category_id', 'category_top_id', 'fullDirNamePath']).count()
category_count.index

MultiIndex([(      '1001', '10', ...),
            (    '100101', '10', ...),
            (  '10010101', '10', ...),
            (  '10010102', '10', ...),
            ('1001010201', '10', ...),
            ('1001010202', '10', ...),
            (  '10010103', '10', ...),
            (  '10010104', '10', ...),
            (  '10010105', '10', ...),
            (    '100102', '10', ...),
            ...
            (   '9020401',  '9', ...),
            (   '9020402',  '9', ...),
            (     '90205',  '9', ...),
            (     '90206',  '9', ...),
            (   '9020601',  '9', ...),
            (   '9020602',  '9', ...),
            (   '9020603',  '9', ...),
            (   '9020604',  '9', ...),
            (   '9020605',  '9', ...),
            (     '90207',  '9', ...)],
           names=['category_id', 'category_top_id', 'fullDirNamePath'], length=1951)

In [61]:
category_index = {}
for x in category_count.index:
    category_index[x[0]] = x[1] 
    category_index[x[0]] = x[2] 
    
category_index

{'1001': 'Q&A > 스포츠, 레저 > 구기스포츠',
 '100101': 'Q&A > 스포츠, 레저 > 구기스포츠, 레저 > 축구',
 '10010101': 'Q&A > 스포츠, 레저 > 구기스포츠, 레저 > 축구 > 월드컵, 국가대항전',
 '10010102': 'Q&A > 스포츠, 레저 > 구기스포츠, 레저 > 축구 > 프로축구',
 '1001010201': 'Q&A > 스포츠, 레저 > 구기스포츠, 레저 > 축구 > 프로축구 > 국내프로축구',
 '1001010202': 'Q&A > 스포츠, 레저 > 구기스포츠, 레저 > 축구 > 프로축구 > 해외프로축구',
 '10010103': 'Q&A > 스포츠, 레저 > 구기스포츠, 레저 > 축구 > 아마추어축구',
 '10010104': 'Q&A > 스포츠, 레저 > 구기스포츠, 레저 > 축구 > 축구 기술, 규칙',
 '10010105': 'Q&A > 스포츠, 레저 > 구기스포츠, 레저 > 축구 > 축구 선수, 감독',
 '100102': 'Q&A > 스포츠, 레저 > 구기스포츠, 레저 > 야구',
 '10010201': 'Q&A > 스포츠, 레저 > 구기스포츠, 레저 > 야구 > 국가대항전',
 '10010202': 'Q&A > 스포츠, 레저 > 구기스포츠, 레저 > 야구 > 프로야구',
 '1001020201': 'Q&A > 스포츠, 레저 > 구기스포츠, 레저 > 야구 > 프로야구 > 국내프로야구',
 '1001020202': 'Q&A > 스포츠, 레저 > 구기스포츠, 레저 > 야구 > 프로야구 > 해외프로야구',
 '1001020204': 'Q&A > 스포츠, 레저 > 구기스포츠, 레저 > 야구 > 프로야구 > 두산베어스',
 '1001020205': 'Q&A > 스포츠, 레저 > 구기스포츠, 레저 > 야구 > 프로야구 > 롯데자이언츠',
 '1001020206': 'Q&A > 스포츠, 레저 > 구기스포츠, 레저 > 야구 > 프로야구 > 삼성라이온즈',
 '1001020207': 'Q&A > 스포츠

In [37]:
detail_list = dump_data(
    index_list=index['detail'], 
    sources=['category', 'question', 'detail_question', 'document_id', 'answer'],
)

In [43]:
import pickle

with open('data/naver-kin.detail_list.pkl', 'wb') as fp:
    pickle.dump(detail_list, fp)

In [38]:
detail_df = pd.DataFrame(detail_list)

In [44]:
import pickle

with open('data/naver-kin.detail_df.pkl', 'wb') as fp:
    pickle.dump(detail_df, fp)

In [40]:
detail_df.head()

,detail_question,answer,question,category,document_id
0,페덱스란 기업에 관심이 생겨서 아주아주 소량이라도 주식을 사고 싶어졌습니다주식 1도...,"[안녕하세요,해외주식에 관심이 있으신가 봅니다.해외주식 투자에는 수수료도 비싸지만 ...",[질문] 페덱스 주식을 사려면 어떻게 해야하나요?,"금융 주식, 증권",4-40102-288569597
1,"주식A가 앞으로 5년간 예상되는 배당액이 매년4,000원이고 5년 후에 예상되는 주...",[안녕하세요재테크는 돈테크만돈테크만 입니다.1년마다 4000원의 수익이 발생5년동안...,[질문] 주식 수익률,"금융 주식, 증권",4-40102-250335463
2,금시세를 볼려고 하는데요..금거래소 등 사이트를 들어가봐도제가 볼줄을 몰라서 그런지...,"[안녕하세요,오늘 기준 금시세는1돈에 207,900 원 이었습니다.오늘의 금시세는 ...",[질문] 금시세 확인,"금융 주식, 증권",4-40102-277735807
3,교재를 읽어도 워낙 개념들이 꼬여있고 쓰이는 단어들도 어렵고;;; 개념이 손에 잡히...,"[안녕하세요,사전적으로 어렵게 설명해봐야 아무런 의미가 없으므로 최대한 쉽게 설명애...",[질문] 유가증권이 정확히 뭐지요?,"금융 주식, 증권",4-40102-253179420
4,들어보니 주식을 살때 팔때 수수료가 든다고 하드라구욤....어디가 제일 좋나요?저는...,"[안녕하세요,주식투자에 관심을 가지고 질문을 하신 것 같습니다.예전의 제모습이 생각...",[질문] 주식을 할려면 금융? 그걸 만들어야되잖아요,"금융 주식, 증권",4-40102-298576080


In [41]:
detail_df.reset_index().drop(columns='index').to_json(
    'data/naver-kin.detail_list.json.bz2',
    force_ascii=False,
    compression='bz2',
    orient='records',
    lines=True,
)

MemoryError: 

In [ ]:
import pandas as pd

detail_df = pd.read_json(
    'data/naver-kin.detail_list.json.bz2',
    compression='bz2',
    orient='records',
    lines=True,
)

detail_df.head()

In [65]:
def test(x):
    c_id = x.split('-')[-2]
    if c_id in category_index:
        return category_index[c_id]

    c_id = x.split('-')[0]
    if c_id in category_index:
        return category_index[c_id]
    
    return ''

detail_df['dir_name'] = detail_df['document_id'].apply(lambda x: test(x))

detail_df.head()

,detail_question,answer,question,category,document_id,dir_name
0,페덱스란 기업에 관심이 생겨서 아주아주 소량이라도 주식을 사고 싶어졌습니다주식 1도...,"[안녕하세요,해외주식에 관심이 있으신가 봅니다.해외주식 투자에는 수수료도 비싸지만 ...",[질문] 페덱스 주식을 사려면 어떻게 해야하나요?,"금융 주식, 증권",4-40102-288569597,"Q&A > 경제 > 금융 > 주식, 증권"
1,"주식A가 앞으로 5년간 예상되는 배당액이 매년4,000원이고 5년 후에 예상되는 주...",[안녕하세요재테크는 돈테크만돈테크만 입니다.1년마다 4000원의 수익이 발생5년동안...,[질문] 주식 수익률,"금융 주식, 증권",4-40102-250335463,"Q&A > 경제 > 금융 > 주식, 증권"
2,금시세를 볼려고 하는데요..금거래소 등 사이트를 들어가봐도제가 볼줄을 몰라서 그런지...,"[안녕하세요,오늘 기준 금시세는1돈에 207,900 원 이었습니다.오늘의 금시세는 ...",[질문] 금시세 확인,"금융 주식, 증권",4-40102-277735807,"Q&A > 경제 > 금융 > 주식, 증권"
3,교재를 읽어도 워낙 개념들이 꼬여있고 쓰이는 단어들도 어렵고;;; 개념이 손에 잡히...,"[안녕하세요,사전적으로 어렵게 설명해봐야 아무런 의미가 없으므로 최대한 쉽게 설명애...",[질문] 유가증권이 정확히 뭐지요?,"금융 주식, 증권",4-40102-253179420,"Q&A > 경제 > 금융 > 주식, 증권"
4,들어보니 주식을 살때 팔때 수수료가 든다고 하드라구욤....어디가 제일 좋나요?저는...,"[안녕하세요,주식투자에 관심을 가지고 질문을 하신 것 같습니다.예전의 제모습이 생각...",[질문] 주식을 할려면 금융? 그걸 만들어야되잖아요,"금융 주식, 증권",4-40102-298576080,"Q&A > 경제 > 금융 > 주식, 증권"


In [66]:
detail_df = detail_df.fillna('')

In [67]:
detail_df[ detail_df['dir_name'] == '']

,detail_question,answer,question,category,document_id,dir_name
111,주식투자를 시작하려고 준비 중입니다.주식수수료무료인 증권사에서 주식계좌를 개설하려고...,[주식수수료무료이면서 주식계좌만들기 좋은 증권사를 추천 드려 볼게요.대신증권 크레온...,[질문] 주식수수료무료 계좌만들기!,[],8-8-249964998,
358,주식거래를 해보려고 오래 전부터 주식공부를 해왔는데요이번에 주식투자를 시작하려고 증...,[증권수수료를 직접 비교해 보시고 싶으시다면 금융투자협회 전자공시서비스 사이트에서증...,[질문] 주식사는법과 증권수수료!,[],8-8-249477047,
1894,감사함니다 이렇게소견을 나눌수있는시간을주셔서정말감사함니다 그리고힘이되구요 계속대화의...,"네,좋은 결과 얻으시구요행복한 하루 되세요^^",[질문] 감사함니다 이렇게소견을 나눌수있는시간을주셔서정말감사함니다 그리고힘이되구요 ...,[],1-1-236530438,
2509,"이번에 아파트를 구입하면서, 제 명의의 아파트를 처분하고 새로 구입하는 아파트를 집...",안녕하세요..증여세는 10년간 합산하여 과세되는 세목이므로그 기간내에 증여할 계획이...,[질문] 증여세 신고방법,[],8-8-257849032,
3566,얼마 전부터 주식에 대한 관심을 갖기 시작하면서 주식투자를 해보기로 결정했습니다.먼...,[주식에 대해 공부를 하셔서 어느 정도의 지식이 쌓이시면 주식하는방법은 크게 어렵지...,[질문] 주식하는방법 알아봐요!,[],8-8-252228445,
8530,주식하는방법좀 알려주세여..ㅠㅠ 요즘 주변에서 은행이자보다 주식하는게 훨씬 이익이라...,[주식하는방법 어렵지 않습니다^^우선 가장먼저 주거래 은행 가셔서 증권계좌 개설하시...,[질문] 주식하는방법 궁금합니다ㅠ 내공50,[],8-8-257140733,
12291,03년에 매입당시의 정확한 취득가액을 알려면 어디서 어떻게 알아보아야 되는것인지 알...,안녕하세요.. 2003년도에 실제로 취득한 가액은매매계약서에 의한 가액입니다.그 매...,[질문] 추가 문의 드립니다,[],1-1209150302-142523177,
14844,이번에 막 성인이 된 20살 여학생입니다.주식공부에 관심이 생겨서 주식공부 좀 하려...,[안녕하세요^^ 이제 막 주식공부를 시작하시는군요~제 생각에는 관련 서적도 많이 읽...,[질문] 주식공부 어떻게 시작해야 할까요?,[],8-8-248977261,
28792,2억 1천 8백에 분양받은 24평 아파트를 3억 5천에 팔 경우 양도세는얼마 일까요...,"[시골의 주택은 거래가격이 많지 않을 것으로, 아버지에게 증여를 하여도, 증여세의 ...",[질문] 양도세...,[],4-4-143879817,
29220,가격대가 좋고 성능이 좋은 노트북 추천좀요.비싼것 말고 가격이 착하고 성능까지 착한...,[저는 지금 LG그램 13인치를 사용하고 있습니다어떤 목적으로 사용하시는지는 모르지...,[질문] 저렴하고 성능좋은 노트북 추천부탁드립니다.,[],8-8-262180594,


In [68]:
import pickle

with open('data/naver-kin.detail_df.pkl', 'wb') as fp:
    pickle.dump(detail_df, fp)

In [88]:
len(detail_df)

2719387

In [87]:
pd.options.display.float_format = '{:,}'.format
pd.options.display.max_rows = 2000

c_count_df = detail_df.groupby(by='dir_name').size().to_frame()
c_count_df

,0
dir_name,
,15517
Q&A > 건강 > 건강상담,4195
Q&A > 건강 > 건강상담 > 가정의학과,3224
Q&A > 건강 > 건강상담 > 내과,6624
Q&A > 건강 > 건강상담 > 내과 > 감염내과,2515
Q&A > 건강 > 건강상담 > 내과 > 내분비대사내과,1729
Q&A > 건강 > 건강상담 > 내과 > 류마티스내과,154
Q&A > 건강 > 건강상담 > 내과 > 소화기내과,14091
Q&A > 건강 > 건강상담 > 내과 > 순환기내과,2798


In [ ]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_prop = fm.FontProperties(fname=path, size=12)

plt.rcParams["font.family"] = font_prop.get_name()
plt.rcParams["font.size"] = 12
plt.rcParams["figure.figsize"] = (20, 10)

c_count_df.plot(kind='barh', figsize=(15, 10))